In [1]:
import findspark

findspark.init("D:/spark/spark")

In [9]:
from pyspark import SparkConf, SparkContext, SQLContext
import re
from operator import add
import io 
import spark

In [3]:
conf = (SparkConf ()
        .setMaster("local[8]")
        .setAppName("sample app for reading files")
        .set("spark.executor.memory", "2g"))

sc = SparkContext.getOrCreate(conf=conf)


In [14]:
sqlContext = SQLContext (sc)
df = sqlContext.read.option("header", "false").csv("file[1-2]*")
df.show()

+--------------------+
|                 _c0|
+--------------------+
|cat mat rat cat m...|
|         lat mat cat|
|                 sat|
|                 and|
|the or and cat ma...|
|             cat sat|
|                 mad|
|                 xie|
|        sat mat mon |
|      tues wed thurs|
+--------------------+



In [51]:
def mapper(row):
    # get the text from the row entry
    text = row._c0
     
    #remove unwanted chars
    text=re.sub("\\W"," ",text)

    # lower case text and split by space to get the words
    words = text.lower ().split (" ")

    # for each word, send back a count of 1
    return [[w, 1] for w in words]


def reducer1(df):
    df.show (2, False)

    mapped_rdd = df.rdd.flatMap (lambda row: mapper (row))
    print("Mapper output -")
    print(mapped_rdd.collect())
   
    counts_rdd = mapped_rdd.reduceByKey (lambda a,b:b)
    
    print("Reducer 1 output -")
    
    print(counts_rdd.collect())
    
    word_count = reducer2(counts_rdd)
    
    return word_count

def reducer2(counts_rdd):
    return counts_rdd.values().sum()

In [52]:
print("distinct count =",reducer1(df) )

+-----------------------+
|_c0                    |
+-----------------------+
|cat mat rat cat mat sat|
|lat mat cat            |
+-----------------------+
only showing top 2 rows

Mapper output -
[['cat', 1], ['mat', 1], ['rat', 1], ['cat', 1], ['mat', 1], ['sat', 1], ['lat', 1], ['mat', 1], ['cat', 1], ['sat', 1], ['and', 1], ['the', 1], ['or', 1], ['and', 1], ['cat', 1], ['mat', 1], ['vijay', 1], ['cat', 1], ['sat', 1], ['mad', 1], ['xie', 1], ['sat', 1], ['mat', 1], ['mon', 1], ['', 1], ['tues', 1], ['wed', 1], ['thurs', 1]]
Reducer 1 output -
[('cat', 1), ('sat', 1), ('vijay', 1), ('xie', 1), ('mon', 1), ('', 1), ('wed', 1), ('thurs', 1), ('mat', 1), ('rat', 1), ('lat', 1), ('and', 1), ('the', 1), ('or', 1), ('mad', 1), ('tues', 1)]
distinct count = 16
